In [1]:
import gc
import numpy as np
import pandas as pd
import yfinance as yf
import back_testing as bt
import matplotlib.pyplot as plt

gc.collect()

120

In [2]:
tickers = ["GC=F", "^GSPC", "LMT"]
data = yf.download(tickers, start="2000-01-01")

[*********************100%***********************]  3 of 3 completed


In [3]:
# 시각화
plt.figure(figsize=(12, 3), dpi=400)
plt.plot(data['Close'])
plt.title("Gold, S&P 500, LMT")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend(data['Close'].columns, loc='upper left')
plt.tight_layout()
plt.show()

# 시각화
plt.figure(figsize=(12, 3), dpi=400)
plt.plot(data['Close'].pct_change())
plt.title("Gold, S&P 500, LMT")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend(data['Close'].columns, loc='upper left')
plt.tight_layout()
plt.show()

In [4]:
# 각 투자 성향별 배분비율 (Gold, S&P500, LMT 순서)
allocation_ratios = {
    'Very_Conservative': [0.8, 0.15, 0.05],  # 매우 보수적: 금 80%, S&P500 15%, LMT 5%
    'Conservative': [0.4, 0.5, 0.1],         # 보수적: 금 40%, S&P500 50%, LMT 10%
    'Balanced': [0.25, 0.5, 0.25],           # 균형: 금 25%, S&P500 50%, LMT 25%
    'Aggressive': [0.1, 0.3, 0.6],           # 공격적: 금 10%, S&P500 30%, LMT 60%
    'Very_Aggressive': [0.05, 0.15, 0.8]     # 매우 공격적: 금 5%, S&P500 15%, LMT 80%
}

start_date = '2000-01-01'
end_date = '2025-06-30'
date_index = pd.date_range(start=start_date, end=end_date, freq='D')
col_order = list(data['Close'].columns)   # 예: ['Gold', 'S&P500', 'LMT']

# 1) 전략별 상수 가중치 테이블들 (weight_dict) — vectorized
weight_dict = {}
for strategy, ratios in allocation_ratios.items():
    arr = np.tile(np.asarray(ratios, dtype=float), (len(date_index), 1))
    strategy_df = pd.DataFrame(arr, index=date_index, columns=col_order)
    weight_dict[strategy] = strategy_df

In [5]:
# Calculate S&P 500 200-day moving average and market conditions
# Add VIX data for market volatility analysis
vix_data = yf.download("^VIX", start="2000-01-01")['Close']

sp500_data = data['Close']['^GSPC'].dropna()
sp500_ma200 = sp500_data.rolling(window=200).mean()

# Create market condition indicators
market_conditions = pd.DataFrame(index=sp500_data.index)
market_conditions['sp500_price'] = sp500_data
market_conditions['sp500_ma200'] = sp500_ma200
market_conditions['above_ma200'] = sp500_data > sp500_ma200
market_conditions['vix'] = vix_data.reindex(sp500_data.index, method='ffill')
market_conditions['vix_high'] = market_conditions['vix'] >= 20

# Create combined market condition
market_conditions['market_regime'] = 'Balanced'  # Default
market_conditions.loc[
    (market_conditions['above_ma200'] == True) & (market_conditions['vix_high'] == False), 
    'market_regime'
] = 'Very_Aggressive'
market_conditions.loc[
    (market_conditions['above_ma200'] == False) & (market_conditions['vix_high'] == True), 
    'market_regime'
] = 'Very_Conservative'

regimes = (
    market_conditions
    .reindex(date_index)[ 'market_regime' ]
    .ffill()
)

#   - 레짐 -> 리스트 매핑 (모르면 Balanced로 대체)
regime_weights_list = regimes.map(
    lambda r: allocation_ratios.get(r, allocation_ratios['Balanced'])
).to_list()

dynamic_weights = pd.DataFrame(regime_weights_list, index=date_index, columns=col_order)

weight_dict['Dynamic'] = dynamic_weights

[*********************100%***********************]  1 of 1 completed


In [34]:
# 백테스팅 설정
base_fee = 0.0002  # 기본 수수료 0.02%
turnover_multiplier = 1  # turnover에 따른 수수료 증가 배수
rebalance_freq = 'Q'  # 리밸런싱 주기: 'D', 'W', 'M', 'Q', 'Y'

print(f"설정:")
print(f"  - 기본 수수료: {base_fee*100:.2f}%")
print(f"  - Turnover 배수: {turnover_multiplier}")
print(f"  - 리밸런싱 주기: {rebalance_freq}")

# 백테스팅 실행
result_df = pd.DataFrame()
for str_name in weight_dict.keys():
    result_df = bt.return_Table(
        weight_dict[str_name], 
        data['Close'],  # 가격 데이터 직접 전달
        result_df,
        rebalance_freq=rebalance_freq,
        fee=base_fee,
        turnover_multiplier=turnover_multiplier
    )
    
# 컬럼명 설정 (S&P 500 + 전략들)
result_df.columns = ['S&P 500'] + list(weight_dict.keys())

bt.plot_result(result_df)

[*********************100%***********************]  1 of 1 completed

설정:
  - 기본 수수료: 0.02%
  - Turnover 배수: 1
  - 리밸런싱 주기: Q


In [16]:
for freq in ['D', 'W', 'M', 'Q', 'Y']:  
    # turnover 계산 (예: 월별 리밸런스)
    turnover_per_period = bt.cal_turnover(weight_dict['Dynamic'], rebalance_freq=freq)

    # 수수료 효과 계산
    fee_log = bt.cal_dynamic_fee(turnover_per_period, fee=0.0002, turnover_multiplier=1)

    # 총 수수료 (log-return 기준 합산)
    total_fee = fee_log.sum()
    print(f"{freq} turnover_mean: {turnover_per_period.mean():.4f}")
    print(f"{freq} 총 수수료 (log return 합): {total_fee:.4f}")


D turnover_mean: 0.0296
D 총 수수료 (log return 합): -0.1544
W turnover_mean: 0.0914
W 총 수수료 (log return 합): -0.0676
M turnover_mean: 0.1301
M 총 수수료 (log return 합): -0.0216
Q turnover_mean: 0.2108
Q 총 수수료 (log return 합): -0.0111
Y turnover_mean: 0.3577
Y 총 수수료 (log return 합): -0.0047


In [36]:
metric = bt.METRICS(result_df.dropna())
metric.cal_describe()
metric.cal_monthly_statistics()

In [27]:
metric.annual_statistics

S&P 500  \
Date                            2000-12-31 ~ 2025-09-30   
count                                               100   
cumulative return                             363.40923   
annualized return mean                          0.06326   
annualized return std                           0.18395   
annualized return downside std                  0.15065   
sharpe ratio                                    0.34388   
sortino ratio                                   0.41991   
gross profit                                    4.16312   
gross loss                                     -2.62968   
profit factor                                   1.58313   
maximum drawdown                               -0.47741   
calmar ratio                                     0.1325   
profitable years                                     18   
unprofitable years                                    8   
t-statistic                                         0.0   
p_value                                             1.0   

                                      Very_Conservative  \
Date                            2000-12-31 ~ 2025-09-30   
count                                               100   
cumulative return                            1349.36429   
annualized return mean                          0.11288   
annualized return std                           0.12336   
annualized return downside std                  0.07584   
sharpe ratio                                    0.91504   
sortino ratio                                   1.48835   
gross profit                                    3.95172   
gross loss                                     -1.27801   
profit factor                                   3.09209   
maximum drawdown                               -0.23076   
calmar ratio                                    0.48914   
profitable years                                     20   
unprofitable years                                    6   
t-statistic                                     2.24029   
p_value                                         0.02618   

                                           Conservative  \
Date                            2000-12-31 ~ 2025-09-30   
count                                               100   
cumulative return                            1658.13287   
annualized return mean                          0.12151   
annualized return std                           0.13424   
annualized return downside std                  0.08074   
sharpe ratio                                    0.90517   
sortino ratio                                   1.50489   
gross profit                                    4.26008   
gross loss                                     -1.39324   
profit factor                                   3.05767   
maximum drawdown                                -0.2133   
calmar ratio                                    0.56966   
profitable years                                     24   
unprofitable years                                    2   
t-statistic                                      2.5579   
p_value                                         0.01128   

                                               Balanced  \
Date                            2000-12-31 ~ 2025-09-30   
count                                               100   
cumulative return                            1392.60306   
annualized return mean                          0.11419   
annualized return std                           0.14148   
annualized return downside std                  0.10089   
sharpe ratio                                    0.80706   
sortino ratio                                   1.13174   
gross profit                                     4.2778   
gross loss                                     -1.57469   
profit factor                                   2.71659   
maximum drawdown                               -0.27727   
calmar ratio                                    0.41182   
profitable years                 

In [10]:
metric.monthly_statistics

S&P 500 Very_Conservative Conservative  Balanced  \
Mean                 0.00023           0.00043      0.00047   0.00044   
Standard deviation    0.0122           0.00917      0.00948   0.00993   
Standard error       0.00015           0.00012      0.00012   0.00013   
t-statistic           1.6447            3.6966       4.0402    3.7255   
Min                 -0.12765          -0.08244     -0.09116  -0.10291   
25%                 -0.00468          -0.00404     -0.00413  -0.00415   
50%                  0.00065           0.00066      0.00069   0.00063   
75%                  0.00582           0.00549      0.00544   0.00547   
Max                  0.10957           0.06993      0.08944   0.07481   
Skew                -0.35349          -0.34107     -0.38958  -0.52773   
Kurtosis            10.98262           5.60944      9.08585  10.07041   

                   Aggressive Very_Aggressive  Dynamic  
Mean                  0.00035         0.00029  0.00071  
Standard deviation    0.01039         0.01105  0.00842  
Standard error        0.00013         0.00014  0.00011  
t-statistic            2.9226          2.3274   6.7947  
Min                  -0.11819        -0.12292 -0.06244  
25%                  -0.00402        -0.00433 -0.00334  
50%                   0.00066         0.00066  0.00082  
75%                   0.00529         0.00535  0.00512  
Max                   0.09115          0.0968  0.06993  
Skew                 -0.58511        -0.48049 -0.13452  
Kurtosis             11.90647        11.93579  6.12885

In [37]:
metric.save_results('./general_results')